In [94]:
import pandas as pd
import numpy as np
import string
import json
table = str.maketrans('', '', string.punctuation)
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
Stemmer=SnowballStemmer("english", ignore_stopwords=True)
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [85]:
def word_count(str):
    counts = dict()
    str=remove_stopwords(str)
    words = word_tokenize(str)
    #removing stop words
    for word in words:
        #word = Stemmer.stem(word)
        word= wordnet_lemmatizer.lemmatize(word)
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

In [86]:
with open('../data/raw/WikiPassageQA/document_passages.json') as f:
    data = json.load(f)

In [87]:
df = pd.read_csv("../data/raw/extracted_query_data/train_exp.csv")

In [109]:
df

,DocumentID,DocumentName,QID,Question,RelevantPassages
0,672,Evangelicalism.html,3086,What is the role of conversionism in Evangelic...,4
1,359,French_Revolution.html,195,How did the assault on the Bastille the first ...,1
2,359,French_Revolution.html,195,How did the assault on the Bastille the first ...,2
3,285,Albania.html,557,What is the prehistory of Albania?,4
4,579,Central_Powers.html,1508,What significance did Bulgaria have in the end...,14
...,...,...,...,...,...
5555,641,Ancient_Egypt.html,1971,How did conquering of Alexander the great effe...,29
5556,641,Ancient_Egypt.html,1971,How did conquering of Alexander the great effe...,30
5557,740,Amnesty_International.html,1153,What is Amnesty International?,0
5558,740,Amnesty_International.html,1153,What is Amnesty International?,9


In [88]:
docid=str(df.iloc[0][0])
relp = str(df.iloc[0][4])
question=df.iloc[0][3].translate(table).lower()
passage=data[docid][relp].translate(table).lower()

In [89]:
q = word_count(question)
p = word_count(passage)
setA = set(q)
setB = set(p)
print("The overlapping words between question and passage is:- ")
print(setA.intersection(setB))
print("Total number of overlapping words: "+str(len(setA.intersection(setB))))
print("Counts of the overlapping words in question and passage:-")
for item in setA.intersection(setB):
    print("Q count for the word '"+ item +"' is "+str(q[item]))
    print("P count for the word '"+ item +"' is "+str(p[item]))

The overlapping words between question and passage is:- 
{'evangelicalism', 'conversionism'}
Total number of overlapping words: 2
Counts of the overlapping words in question and passage:-
Q count for the word 'evangelicalism' is 1
P count for the word 'evangelicalism' is 1
Q count for the word 'conversionism' is 1
P count for the word 'conversionism' is 1


In [100]:
counts=[]
Qcounts=[]
Pcounts=[]
for i in range(len(df)):
    docid=str(df.iloc[i][0])
    relp = str(df.iloc[i][4])
    question=df.iloc[i][3].translate(table).lower()
    passage=data[docid][relp].translate(table).lower()
    q = word_count(question)
    p = word_count(passage)
    setA = set(q)
    setB = set(p)
    intersect = setA.intersection(setB)
    resQ = {key: q[key] for key in intersect} 
    resP = {key: p[key] for key in intersect}
    Qcounts.append(resQ)
    Pcounts.append(resP)
    counts.append(len(intersect))

In [101]:
counts = np.array(counts)
counts.mean(),counts.std()

(2.427697841726619, 1.4443473627849468)

In [120]:
list(Qcounts[10].keys()),list(Pcounts[10].values())

(['birth', 'considered', 'rate'], [4, 1, 4])

In [130]:
allquestions = list(df["Question"])
allpassages=[]
for i in data.keys():
    for j in data[i].keys():
        allpassages.append(data[i][j])
allsent = allpassages+allquestions        

In [129]:
vectorizer = CountVectorizer()
X = vectorizer.fit(allsent)